In [1]:
import os, re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional
from keras.layers import GlobalMaxPool1D, Conv1D, MaxPooling1D, Flatten, GRU
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import AUC
from keras.models import load_model

Using TensorFlow backend.


# Open and save data and weights

In [2]:
x_tr_1 = np.genfromtxt('data_preproc/x_tr_1.csv')

In [3]:
x_tr_1.shape

(159571, 200)

In [4]:
x_ts_1 = np.genfromtxt('data_preproc/x_ts_1.csv')

In [5]:
x_ts_1.shape

(153164, 200)

In [6]:
x_tr_2 = np.genfromtxt('data_preproc/x_tr_2.csv')

In [7]:
x_tr_2.shape

(159571, 200)

In [8]:
x_ts_2 = np.genfromtxt('data_preproc/x_ts_2.csv')

In [9]:
x_ts_2.shape

(153164, 200)

In [10]:
x_tr_3 = np.genfromtxt('data_preproc/x_tr_3.csv')

In [11]:
x_tr_3.shape

(159571, 200)

In [12]:
x_ts_3 = np.genfromtxt('data_preproc/x_ts_3.csv')

In [13]:
x_ts_3.shape

(153164, 200)

In [14]:
x_tr_4 = np.genfromtxt('data_preproc/x_tr_4.csv')

In [15]:
x_tr_4.shape

(159571, 200)

In [16]:
x_ts_4 = np.genfromtxt('data_preproc/x_ts_4.csv')

In [17]:
x_ts_4.shape

(153164, 200)

In [18]:
emb_matrix_glove = np.genfromtxt('data_preproc/emb_matr_glove.csv')

In [19]:
emb_matrix_glove.shape

(30000, 200)

In [20]:
emb_matrix_ft = np.genfromtxt('data_preproc/emb_matr_ft.csv')

In [21]:
emb_matrix_ft.shape

(30000, 200)

In [22]:
train = pd.read_csv('data/train.csv/train.csv')
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y_tr = train[col].values

# Build the model

In [23]:
max_len = 200
max_feature = 30000

In [24]:
def build_model(emb_matrix):
    """
    Questa funzione crea il modello e lo compila
    return: a model
    
    emb_matrix: matrice che inizializza i pesi dell'Embedding layer
    
    """
    
    inp = Input(shape=(None,))
    emb = Embedding(max_feature, 200, input_length=max_len, weights=[emb_matrix])(inp)
    lstm = LSTM(units=60, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)(emb)
    fl = Flatten()(lstm)
    d1 = Dense(100, activation='relu')(fl)
    out = Dense(6, activation='sigmoid')(d1)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='acc')])
    
    return model 

In [25]:
cp = ModelCheckpoint('model_/best_lstm.h5', monitor='val_acc', verbose=1, save_best_only=True)
es = EarlyStopping(verbose=1)
callbacks = [cp, es]

In [26]:
# training with x_1: without punctuation and digits

model1 = build_model(emb_matrix_glove)

history1_1 = model1.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 253s 2ms/step - loss: 0.0672 - acc: 0.9617 - val_loss: 0.0519 - val_acc: 0.9797

Epoch 00001: val_acc improved from -inf to 0.97971, saving model to model_/best_lstm.h5
Epoch 2/3
143613/143613 [==============================] - 252s 2ms/step - loss: 0.0465 - acc: 0.9832 - val_loss: 0.0472 - val_acc: 0.9814

Epoch 00002: val_acc improved from 0.97971 to 0.98144, saving model to model_/best_lstm.h5
Epoch 3/3
143613/143613 [==============================] - 244s 2ms/step - loss: 0.0403 - acc: 0.9880 - val_loss: 0.0476 - val_acc: 0.9775

Epoch 00003: val_acc did not improve from 0.98144
Epoch 00003: early stopping


In [27]:
# training with x_2: without URLs, abbreviations and tags

model2 = build_model(emb_matrix_glove)

history1_2 = model2.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 216s 2ms/step - loss: 0.0667 - acc: 0.9621 - val_loss: 0.0525 - val_acc: 0.9782

Epoch 00001: val_acc did not improve from 0.98144
Epoch 2/3
143613/143613 [==============================] - 216s 2ms/step - loss: 0.0469 - acc: 0.9832 - val_loss: 0.0496 - val_acc: 0.9787

Epoch 00002: val_acc did not improve from 0.98144
Epoch 3/3
143613/143613 [==============================] - 231s 2ms/step - loss: 0.0414 - acc: 0.9876 - val_loss: 0.0515 - val_acc: 0.9725

Epoch 00003: val_acc did not improve from 0.98144
Epoch 00003: early stopping


In [28]:
# training with x_3: with lemmatization

model3 = build_model(emb_matrix_glove)

history1_3 = model3.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 244s 2ms/step - loss: 0.0661 - acc: 0.9630 - val_loss: 0.0528 - val_acc: 0.9777

Epoch 00001: val_acc did not improve from 0.98144
Epoch 2/3
143613/143613 [==============================] - 249s 2ms/step - loss: 0.0479 - acc: 0.9822 - val_loss: 0.0511 - val_acc: 0.9774

Epoch 00002: val_acc did not improve from 0.98144
Epoch 3/3
143613/143613 [==============================] - 249s 2ms/step - loss: 0.0429 - acc: 0.9859 - val_loss: 0.0501 - val_acc: 0.9789

Epoch 00003: val_acc did not improve from 0.98144


In [29]:
# training with x_4: with stemming

model4 = build_model(emb_matrix_glove)

history1_4 = model4.fit(x_tr_4, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 233s 2ms/step - loss: 0.0728 - acc: 0.9499 - val_loss: 0.0610 - val_acc: 0.9655

Epoch 00001: val_acc did not improve from 0.98144
Epoch 2/3
143613/143613 [==============================] - 250s 2ms/step - loss: 0.0558 - acc: 0.9728 - val_loss: 0.0571 - val_acc: 0.9715

Epoch 00002: val_acc did not improve from 0.98144
Epoch 3/3
143613/143613 [==============================] - 254s 2ms/step - loss: 0.0516 - acc: 0.9771 - val_loss: 0.0578 - val_acc: 0.9687

Epoch 00003: val_acc did not improve from 0.98144
Epoch 00003: early stopping


In [30]:
# training with x_1: without punctuations and digits

model1_2 = build_model(emb_matrix_ft)

history_1 = model1_2.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 255s 2ms/step - loss: 0.0657 - acc: 0.9628 - val_loss: 0.0503 - val_acc: 0.9833

Epoch 00001: val_acc improved from 0.98144 to 0.98328, saving model to model_/best_lstm.h5
Epoch 2/3
143613/143613 [==============================] - 259s 2ms/step - loss: 0.0455 - acc: 0.9840 - val_loss: 0.0466 - val_acc: 0.9820

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 234s 2ms/step - loss: 0.0397 - acc: 0.9886 - val_loss: 0.0478 - val_acc: 0.9790

Epoch 00003: val_acc did not improve from 0.98328
Epoch 00003: early stopping


In [31]:
# training with x_2: without URLs, abbreviations and tags

model2_2 = build_model(emb_matrix_ft)

history_2 = model2_2.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 239s 2ms/step - loss: 0.0647 - acc: 0.9641 - val_loss: 0.0513 - val_acc: 0.9776

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 234s 2ms/step - loss: 0.0456 - acc: 0.9842 - val_loss: 0.0481 - val_acc: 0.9803

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 233s 2ms/step - loss: 0.0396 - acc: 0.9887 - val_loss: 0.0477 - val_acc: 0.9798

Epoch 00003: val_acc did not improve from 0.98328


In [32]:
# training with x_3: with lemmatization

model3_2 = build_model(emb_matrix_ft)

history_3 = model3_2.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 238s 2ms/step - loss: 0.0662 - acc: 0.9616 - val_loss: 0.0531 - val_acc: 0.9751

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 257s 2ms/step - loss: 0.0470 - acc: 0.9826 - val_loss: 0.0500 - val_acc: 0.9794

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 261s 2ms/step - loss: 0.0416 - acc: 0.9871 - val_loss: 0.0511 - val_acc: 0.9779

Epoch 00003: val_acc did not improve from 0.98328
Epoch 00003: early stopping


In [33]:
# training with x_4: with stemming

model4_2 = build_model(emb_matrix_ft)

history_4 = model4_2.fit(x_tr_4, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 260s 2ms/step - loss: 0.0715 - acc: 0.9530 - val_loss: 0.0631 - val_acc: 0.9641

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 260s 2ms/step - loss: 0.0550 - acc: 0.9738 - val_loss: 0.0572 - val_acc: 0.9705

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 260s 2ms/step - loss: 0.0512 - acc: 0.9775 - val_loss: 0.0560 - val_acc: 0.9716

Epoch 00003: val_acc did not improve from 0.98328


# Model without embedding matrix

In [34]:
def build_model_():
    
    inp = Input(shape=(None,))
    emb = Embedding(max_feature, 200, input_length=max_len)(inp)
    lstm = LSTM(units=60, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)(emb)
    fl = Flatten()(lstm)
    d1 = Dense(100, activation='relu')(fl)
    out = Dense(6, activation='sigmoid')(d1)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='acc')])
    
    return model

In [35]:
model1_3 = build_model_()

history3_1 = model1_3.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 254s 2ms/step - loss: 0.0645 - acc: 0.9640 - val_loss: 0.0508 - val_acc: 0.9777

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 256s 2ms/step - loss: 0.0446 - acc: 0.9848 - val_loss: 0.0493 - val_acc: 0.9810

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 220s 2ms/step - loss: 0.0378 - acc: 0.9899 - val_loss: 0.0526 - val_acc: 0.9710

Epoch 00003: val_acc did not improve from 0.98328
Epoch 00003: early stopping


In [36]:
model2_3 = build_model_()

history3_2 = model2_3.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 217s 2ms/step - loss: 0.0665 - acc: 0.9607 - val_loss: 0.0536 - val_acc: 0.9751

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 216s 2ms/step - loss: 0.0451 - acc: 0.9845 - val_loss: 0.0515 - val_acc: 0.9757

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 215s 1ms/step - loss: 0.0375 - acc: 0.9899 - val_loss: 0.0540 - val_acc: 0.9718

Epoch 00003: val_acc did not improve from 0.98328
Epoch 00003: early stopping


In [37]:
model3_3 = build_model_()

history3_3 = model3_3.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 220s 2ms/step - loss: 0.0669 - acc: 0.9602 - val_loss: 0.0542 - val_acc: 0.9764

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 230s 2ms/step - loss: 0.0465 - acc: 0.9831 - val_loss: 0.0517 - val_acc: 0.9765

Epoch 00002: val_acc did not improve from 0.98328
Epoch 3/3
143613/143613 [==============================] - 217s 2ms/step - loss: 0.0395 - acc: 0.9885 - val_loss: 0.0553 - val_acc: 0.9697

Epoch 00003: val_acc did not improve from 0.98328
Epoch 00003: early stopping


In [38]:
model4_3 = build_model_()

history3_4 = model4_3.fit(x_tr_4, y_tr, batch_size=32, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 843s 6ms/step - loss: 0.0662 - acc: 0.9603 - val_loss: 0.0581 - val_acc: 0.9715

Epoch 00001: val_acc did not improve from 0.98328
Epoch 2/3
143613/143613 [==============================] - 901s 6ms/step - loss: 0.0537 - acc: 0.9749 - val_loss: 0.0591 - val_acc: 0.9700

Epoch 00002: val_acc did not improve from 0.98328
Epoch 00002: early stopping
